# edges - Biodiversity, GLAM

0. Load data
1. Spatial Overlay: Intersect country boundaries with WWF ecoregions
2. Weight Calculation: Compute the percentage of each country occupied by each ecoregion
3. CF Matching: Merge Scherer et al. CFs based on eco_id
4. Output Generation: Creates a structured dataset ready for edges integration

In [1]:
import geopandas as gpd
import pandas as pd
import json
import country_converter as coco
import bw2data as bd
from shapely.geometry import Point, MultiPoint, GeometryCollection

# 0. Load data

## Biodiversity Characterisation Factors
https://zenodo.org/records/10114493

In [2]:
cfs_country = pd.read_csv("data/CF_domain_country.csv", encoding='latin1')
cfs_country.columns

Index(['objectid', 'iso3cd', 'romnam', 'm49code', 'species_group', 'kingdom',
       'habitat_id', 'habitat', 'CF_occ_avg_reg', 'CF_occ_avg_reg_rsd',
       'CF_occ_avg_glo', 'CF_occ_avg_glo_rsd', 'CF_occ_mar_reg',
       'CF_occ_mar_reg_rsd', 'CF_occ_mar_glo', 'CF_occ_mar_glo_rsd',
       'CF_tra_avg_reg', 'CF_tra_avg_reg_rsd', 'CF_tra_avg_glo',
       'CF_tra_avg_glo_rsd', 'CF_tra_mar_reg', 'CF_tra_mar_reg_rsd',
       'CF_tra_mar_glo', 'CF_tra_mar_glo_rsd', 'quality_reg', 'quality_glo'],
      dtype='object')

In [3]:
cfs = pd.read_csv("data/CF_domain.csv", encoding='latin1')
cfs.columns

Index(['realm', 'biome', 'eco_id', 'eco_name', 'species_group', 'kingdom',
       'habitat_id', 'habitat', 'CF_occ_avg_reg', 'CF_occ_avg_glo',
       'CF_occ_mar_reg', 'CF_occ_mar_glo', 'CF_tra_avg_reg', 'CF_tra_avg_glo',
       'CF_tra_mar_reg', 'CF_tra_mar_glo', 'quality_reg', 'quality_glo'],
      dtype='object')

## Ecoregions
https://www.worldwildlife.org/publications/terrestrial-ecoregions-of-the-world

In [4]:
ecoregions = gpd.read_file("data/official/wwf_terr_ecos.shp")
# Ensure all are in the same CRS (WGS84)
ecoregions = ecoregions.to_crs("EPSG:4326")
# Create eco_id in ecoregions to match with CFs (from float to int)
ecoregions['eco_id'] = ecoregions['ECO_ID'].astype(int)
ecoregions['eco_geometry'] = ecoregions.geometry
ecoregions.columns

Index(['OBJECTID', 'AREA', 'PERIMETER', 'ECO_NAME', 'REALM', 'BIOME',
       'ECO_NUM', 'ECO_ID', 'ECO_SYM', 'GBL_STAT', 'G200_REGIO', 'G200_NUM',
       'G200_BIOME', 'G200_STAT', 'Shape_Leng', 'Shape_Area', 'area_km2',
       'eco_code', 'PER_area', 'PER_area_1', 'PER_area_2', 'geometry',
       'eco_id', 'eco_geometry'],
      dtype='object')

## Countries
https://www.naturalearthdata.com/downloads/10m-cultural-vectors/
We use map units, as we prefer, for example, to distinguish overseas regions of France.
Dataset: Admin 0 – Details --> Download map units (4.77 MB) version 5.1.1

In [5]:
countries = gpd.read_file("data/ne_10m_admin_0_map_units/ne_10m_admin_0_map_units.shp")
# Ensure all are in the same CRS (WGS84)
countries = countries.to_crs("EPSG:4326")

countries_projected = countries.to_crs("ESRI:54009")
countries['area_km2'] = countries_projected.geometry.area / 1e6
country_areas = countries.set_index('ISO_A3')['area_km2'].to_dict()

## Mining facilities and production data
https://zenodo.org/records/7369478

In [6]:
facilities = gpd.read_file("data/mining/jasansky/data/facilities.gpkg")
production = pd.read_csv("data/mining/jasansky/data/commodities.csv")

In [7]:
print(f"Loaded {len(ecoregions)} ecoregions")
print(f"Loaded {len(countries)} countries")
print(f"Loaded {len(cfs)} CF records")
print(f"Loaded {len(facilities['facility_id'].unique())} facilities")
print(f"Loaded {len(production['facility_id'].unique())} facilities with production records")

Loaded 14458 ecoregions
Loaded 298 countries
Loaded 12375 CF records
Loaded 2413 facilities
Loaded 507 facilities with production records


# 1. Mining Data preparation

In [8]:
### Group reserves by facility and select the newest year for each commodity

cols_to_keep = ["commodity", "value_tonnes"]

df_grouped_production = (
    production.groupby(["facility_id", "year"])
    .apply(lambda g: g[cols_to_keep].to_dict(orient="records"), include_groups=False)
    .reset_index(name="prod_data")
)

def get_newest_data(group):
    """
    Select the newest data per commodity for each facility
    """
    group = group.sort_values("year", ascending=False).reset_index(drop=True)
    
    # Start with newest year as base
    newest_year = group.loc[0, 'year']
    combined = {
        item['commodity']: {**item, 'year': newest_year}
        for item in group.loc[0, 'prod_data']
    }
    
    # Add commodities from older years if missing
    for _, row in group.iloc[1:].iterrows():
        year = row["year"]
        for item in row["prod_data"]:
            commodity = item.get("commodity")
            if commodity not in combined:
                combined[commodity] = {**item, "year": year}
    
    return pd.Series({'prod_data': list(combined.values())})

# Apply the function per facility
productionPerFacility = (
    df_grouped_production.groupby('facility_id')
    .apply(get_newest_data, include_groups=False)
    .reset_index()
)

In [9]:
mainFacilities = facilities[facilities['facility_id'].str.endswith("00") & ~facilities.geometry.is_empty].copy()

# Keep only these that have not stopped production
mainFacilities = mainFacilities[(mainFacilities['production_end'].isnull())].copy()

# Drop unnecessary columns
mainFacilities = mainFacilities.drop(columns=[
    'facility_name', 'facility_other_names', 'sub_site_name', 'sub_site_other_names', 'primary_commodity', 'commodities_products', 'production_start', 'GID_0', 'GID_1', 'GID_2', 'GID_3', 'GID_4', 'production_start', 'production_end', 'surface_area_sq_km', 'concession_area_sq_km'
])

# Merge production with whole facilities, only keep those with geometry
facilitiesProduction = mainFacilities.merge(productionPerFacility, on='facility_id', how='inner')

print(len(facilitiesProduction), "whole facilities with production data and geometry")

462 whole facilities with production data and geometry


In [ ]:


def first_point_from_geom(geom):
    if geom is None:
        return None
    try:
        if geom.is_empty:
            return geom
    except Exception:
        return geom

    gtype = geom.geom_type
    if gtype == "MultiPoint":
        # return first non-empty point
        for p in geom.geoms:
            if not p.is_empty:
                return p
        return geom  # fallback
    if gtype == "GeometryCollection":
        for part in geom:
            if part.geom_type == "Point" and not part.is_empty:
                return part
        # fallback to first element if any
        if len(geom) > 0:
            return geom[0]
        return geom
    # if already a Point (or other single geometry), keep as is
    return geom

# inspection before conversion
print("Geometry types before:")
print(facilitiesProduction.geometry.geom_type.value_counts())

# Convert multipoints to first point
facilitiesProduction['geometry'] = facilitiesProduction.geometry.apply(first_point_from_geom)

# Re-wrap as GeoDataFrame to ensure geometry column is recognized
facilitiesProduction = gpd.GeoDataFrame(facilitiesProduction, geometry='geometry', crs=facilitiesProduction.crs)

# inspection after conversion
print("\nGeometry types after:")
print(facilitiesProduction.geometry.geom_type.value_counts())

Geometry types before:
MultiPoint    462
Name: count, dtype: int64

Geometry types after:
Point    462
Name: count, dtype: int64
Index(['facility_id', 'facility_type', 'facility_equipment', 'activity_status',
       'activity_status_year', 'country', 'source_id', 'comment', 'geometry',
       'prod_data'],
      dtype='object')


# 2. Spatial matching

## Join with ecoregions

In [11]:
### Spacially match the facilities to the ecoregion
facility_ecoregions = gpd.sjoin(facilitiesProduction, ecoregions[['eco_id', 'geometry', 'eco_geometry']], how='left', predicate='within')
facility_ecoregions.set_geometry('geometry', inplace=True)
facility_ecoregions.drop(columns=['index_right'], inplace=True)
print(len(facility_ecoregions), "facilities matched to an ecoregion")
facility_ecoregions.columns

462 facilities matched to an ecoregion


Index(['facility_id', 'facility_type', 'facility_equipment', 'activity_status',
       'activity_status_year', 'country', 'source_id', 'comment', 'geometry',
       'prod_data', 'eco_id', 'eco_geometry'],
      dtype='object')

## Join with countries

In [12]:
### Spacially match the facilities to the countries
facilities_country = gpd.sjoin(
    facility_ecoregions,
    countries[["ISO_A3", "NAME", "geometry"]],
    how="left",
    predicate="within",
)

facilities_country.set_geometry("geometry", inplace=True)
facilities_country.drop(columns=['index_right', 'country'], inplace=True)
facilities_country.rename(columns={'NAME': 'country_name'}, inplace=True)
print(len(facilities_country), "facilities matched to a country")
facilities_country.sort_values(by="country_name", inplace=True)

462 facilities matched to a country


# 3. Weight Calculation

In [13]:
materials = pd.read_excel("data/mining/materials_prices.xlsx")
prices_lookup = materials.set_index('material_id')['price/kg'].to_dict()

In [14]:
### Calculate the economic share of each mine in its country

# Explode the list of dicts
df_expanded = facilities_country.explode('prod_data', ignore_index=True)

# Normalize the dict into columns
prod_cols = pd.json_normalize(df_expanded['prod_data'])
df_expanded = df_expanded.drop(columns=['prod_data']).join(prod_cols)

# Get price per commodity
df_expanded['price_per_kg'] = df_expanded['commodity'].map(prices_lookup)

# Revenue per commodity
df_expanded['revenue'] = df_expanded['value_tonnes'] * 1000 * df_expanded['price_per_kg']

# Aggregate revenue per facility and keep geometry
facility_revenue = (
    df_expanded
    .groupby(['facility_id', 'country_name'], as_index=False)
    .agg(
        revenue=('revenue', 'sum'),
        geometry=('geometry', 'first'),
        ISO_A3=('ISO_A3', 'first'),
        eco_id=('eco_id', 'first')
    )
)

# Total revenue per country
facility_revenue['country_total'] = (
    facility_revenue.groupby('country_name')['revenue'].transform('sum')
)

# Share per facility
facility_revenue['share_in_country'] = (
    facility_revenue['revenue'] / facility_revenue['country_total']
)

facilities_final = gpd.GeoDataFrame(facility_revenue, geometry='geometry', crs=facilitiesProduction.crs)
facilities_final

,facility_id,country_name,revenue,geometry,ISO_A3,eco_id,country_total,share_in_country
0,COM00001.00,Canada,2.323432e+08,POINT (-101.87946 54.77482),CAN,50609.0,6.242829e+10,0.003722
1,COM00007.00,Brazil,3.642607e+08,POINT (-43.73907 -19.86773),BRA,60104.0,1.189033e+10,0.030635
2,COM00009.00,Ghana,6.434837e+08,POINT (-2.36132 6.99983),GHA,30111.0,1.797300e+09,0.358028
3,COM00013.00,Sweden,6.821028e+08,POINT (20.96362 67.07507),SWE,80608.0,1.643874e+09,0.414936
4,COM00014.00,Kazakhstan,3.016076e+08,POINT (77.70573 48.62682),KAZ,81318.0,5.591967e+09,0.053936
...,...,...,...,...,...,...,...,...
455,COM01423.00,Kazakhstan,6.104305e+07,POINT (67.45704 47.90149),KAZ,81318.0,5.591967e+09,0.010916
456,COM01425.00,Kazakhstan,3.153913e+08,POINT (67.38472 47.82432),KAZ,81318.0,5.591967e+09,0.056401
457,COM01426.00,Kazakhstan,1.821158e+08,POINT (67.44467 47.86966),KAZ,81318.0,5.591967e+09,0.032567
458,COM01430.00,China,5.507497e+08,POINT (116.40544 25.18979),CHN,40118.0,1.806956e+09,0.304794


In [15]:
aggregated = facilities_final.groupby(["ISO_A3", "eco_id"], as_index=False).agg(
    share_in_country=("share_in_country", "sum"), NAME=("country_name", "first")
)

# Normalize ecoregion shares so they sum to 1 per country
aggregated["share_in_country"] = aggregated["share_in_country"] / aggregated.groupby(
    "ISO_A3"
)["share_in_country"].transform("sum")

country_shares = aggregated.groupby("ISO_A3")["share_in_country"].sum().round(1)
print("Country shares:", country_shares.unique())

Country shares: [1.]


In [16]:
# Create a list with the countries and the total revenue per country
country_totals_list = facilities_final[['ISO_A3', 'country_name', 'country_total']].drop_duplicates().sort_values('country_name').reset_index(drop=True)

# 4. CF matching

## Dataframe matching

In [17]:
facilities_final_cfs = aggregated.merge(
    cfs,
    on='eco_id',
    how='left'
)
facilities_final_cfs.columns

Index(['ISO_A3', 'eco_id', 'share_in_country', 'NAME', 'realm', 'biome',
       'eco_name', 'species_group', 'kingdom', 'habitat_id', 'habitat',
       'CF_occ_avg_reg', 'CF_occ_avg_glo', 'CF_occ_mar_reg', 'CF_occ_mar_glo',
       'CF_tra_avg_reg', 'CF_tra_avg_glo', 'CF_tra_mar_reg', 'CF_tra_mar_glo',
       'quality_reg', 'quality_glo'],
      dtype='object')

## Create final output

In [18]:
output_cols = [
    'ISO_A3',
    'NAME',
    'eco_id',
    'eco_name',
    'share_in_country',
    'species_group',
    'habitat_id',
    'habitat',
    'CF_occ_avg_reg',
    'CF_occ_avg_glo',
    'CF_occ_mar_reg',
    'CF_occ_mar_glo',
    'CF_tra_avg_reg',
    'CF_tra_avg_glo',
    'CF_tra_mar_reg',
    'CF_tra_mar_glo',
    'quality_reg',
    'quality_glo'
]

final_output = facilities_final_cfs[output_cols].copy()

# Rename columns for clarity
final_output.columns = [
    'country_iso',
    'country_name',
    'ecoregion_id',
    'ecoregion_name',
    'weight_factor',
    'species_group',
    'habitat_id',
    'habitat_type',
    'cf_occupation_avg_regional',
    'cf_occupation_avg_global',
    'cf_occupation_marginal_regional',
    'cf_occupation_marginal_global',
    'cf_transformation_avg_regional',
    'cf_transformation_avg_global',
    'cf_transformation_marginal_regional',
    'cf_transformation_marginal_global',
    'quality_regional',
    'quality_global'
]

final_output = final_output[final_output['species_group'].notna()].copy()

In [19]:
final_output = final_output.sort_values(
    ['country_name', 'ecoregion_id', 'species_group', 'habitat_id'],
    ascending=True
).reset_index(drop=True)

In [20]:
final_output.to_csv("output/test_severin/test_severin_eukaryota_country_ecoregion_cfs_scherer.csv", index=False)
final_output.head()

,country_iso,country_name,ecoregion_id,ecoregion_name,weight_factor,species_group,habitat_id,habitat_type,cf_occupation_avg_regional,cf_occupation_avg_global,cf_occupation_marginal_regional,cf_occupation_marginal_global,cf_transformation_avg_regional,cf_transformation_avg_global,cf_transformation_marginal_regional,cf_transformation_marginal_global,quality_regional,quality_global
0,ARG,Argentina,60805.0,Patagonian steppe,0.25276,Eukaryota,2,Cropland_Intense,4.865510e-12,9.763369e-15,3.222034e-11,7.636502e-14,1.995565e-09,4.093404e-12,1.369113e-08,3.296604e-11,proxy_type,proxy_type
1,ARG,Argentina,60805.0,Patagonian steppe,0.25276,Eukaryota,3,Cropland_Light,4.001822e-12,9.220825e-15,2.812956e-11,7.379532e-14,1.654530e-09,3.879176e-12,1.204503e-08,3.193200e-11,proxy_type,proxy_type
2,ARG,Argentina,60805.0,Patagonian steppe,0.25276,Eukaryota,4,Cropland_Minimal,2.701971e-12,5.158975e-15,1.753327e-11,3.998171e-14,1.105285e-09,2.160036e-12,7.429924e-09,1.724309e-11,proxy_type,proxy_type
3,ARG,Argentina,60805.0,Patagonian steppe,0.25276,Eukaryota,5,Managed_forest_Intense,3.903607e-13,5.554039e-16,5.623319e-13,7.904418e-16,1.042471e-10,1.519447e-13,1.501451e-10,2.162240e-13,proxy_intensity,proxy_intensity
4,ARG,Argentina,60805.0,Patagonian steppe,0.25276,Eukaryota,6,Managed_forest_Light,1.900403e-13,2.039752e-16,2.745921e-13,2.917274e-16,5.033896e-11,5.520139e-14,7.272575e-11,7.893160e-14,proxy_intensity,proxy_intensity


In [21]:
# Check a country
spain_check = final_output[final_output['country_iso'] == 'ESP'].groupby('ecoregion_id')['weight_factor'].first().sum()
spain_check

1.0

# 5. Matching habitats to ecoinvent
Matching done according to: https://doi.org/10.1007/s11367-021-02003-y

In [22]:
bd.projects.set_current("bw25_ei311")
bf = bd.Database('biosphere')
sorted([m["categories"] for m in bf if "Transformation, from" in m["name"]])

[]

In [23]:
occupation_flows = {
    'Occupation, annual crop': 'Cropland_Intense',
    'Occupation, annual crop, flooded crop': 'Cropland_Intense',
    'Occupation, annual crop, greenhouse': 'Urban_Light',
    'Occupation, annual crop, irrigated': 'Cropland_Intense',
    'Occupation, annual crop, irrigated, extensive': 'Cropland_Light',
    'Occupation, annual crop, irrigated, intensive': 'Cropland_Intense',
    'Occupation, annual crop, non-irrigated': 'Cropland_Intense',
    'Occupation, annual crop, non-irrigated, extensive': 'Cropland_Minimal',
    'Occupation, annual crop, non-irrigated, intensive': 'Cropland_Intense',
    'Occupation, arable land, unspecified use': 'Cropland_Intense',
    # 'Occupation, arable, conservation tillage (obsolete)': '',
    # 'Occupation, arable, conventional tillage (obsolete)': '',
    # 'Occupation, arable, reduced tillage (obsolete)': '',
    # 'Occupation, bare area (non-use)': '',
    'Occupation, construction site': 'Urban_Intense',
    'Occupation, cropland fallow (non-use)': 'Cropland_Minimal',
    'Occupation, dump site': 'Urban_Intense',
    'Occupation, field margin/hedgerow': 'Cropland_Minimal',
    'Occupation, forest, extensive': 'Pasture_Light',
    'Occupation, forest, intensive': 'Managed_forest_Intense',
    'Occupation, forest, primary (non-use)': 'Managed_forest_Minimal',
    'Occupation, forest, secondary (non-use)': 'Pasture_Light',
    'Occupation, forest, unspecified': 'Managed_forest_Intense',
    'Occupation, grassland, natural (non-use)': 'Pasture_Minimal',
    'Occupation, grassland, natural, for livestock grazing': 'Pasture_Minimal',
    'Occupation, heterogeneous, agricultural': 'Cropland_Minimal',
    'Occupation, industrial area': 'Urban_Intense',
    # 'Occupation, inland waterbody, unspecified': '',
    # 'Occupation, lake, artificial': '',
    # 'Occupation, lake, natural (non-use)': '',
    'Occupation, mineral extraction site': 'Urban_Intense',
    'Occupation, pasture, man made': 'Pasture_Intense',
    'Occupation, pasture, man made, extensive': 'Pasture_Light',
    'Occupation, pasture, man made, intensive': 'Pasture_Intense',
    'Occupation, permanent crop': 'Cropland_Intense',
    'Occupation, permanent crop, irrigated': 'Cropland_Intense',
    'Occupation, permanent crop, irrigated, extensive': 'Cropland_Light',
    'Occupation, permanent crop, irrigated, intensive': 'Cropland_Intense',
    'Occupation, permanent crop, non-irrigated': 'Cropland_Intense',
    'Occupation, permanent crop, non-irrigated, extensive': 'Cropland_Minimal',
    'Occupation, permanent crop, non-irrigated, intensive': 'Cropland_Intense',
    # 'Occupation, river, artificial': '',
    # 'Occupation, river, natural (non-use)': '',
    # 'Occupation, seabed, drilling and mining': '',
    # 'Occupation, seabed, infrastructure': '',
    # 'Occupation, seabed, natural (non-use)': '',
    # 'Occupation, seabed, unspecified': '',
    'Occupation, shrub land, sclerophyllous': 'Managed_forest_Minimal',
    # 'Occupation, snow and ice (non-use)': '',
    'Occupation, traffic area, rail network': 'Urban_Intense',
    'Occupation, traffic area, rail/road embankment': 'Urban_Light',
    'Occupation, traffic area, road network': 'Urban_Intense',
    'Occupation, unspecified': 'Urban_Intense',
    'Occupation, unspecified, natural (non-use)': 'Pasture_Minimal',
    'Occupation, urban, continuously built': 'Urban_Intense',
    'Occupation, urban, discontinuously built': 'Urban_Light',
    'Occupation, urban, green area': 'Urban_Minimal',
    'Occupation, urban/industrial fallow (non-use)': 'Urban_Light',
    # 'Occupation, wetland, coastal (non-use)': '',
    # 'Occupation, wetland, inland (non-use)': '',
}

In [24]:
transformation_to_flows = {
    'Transformation, to annual crop': 'Cropland_Intense',
    'Transformation, to annual crop, flooded crop': 'Cropland_Intense',
    'Transformation, to annual crop, greenhouse': 'Urban_Light',
    'Transformation, to annual crop, irrigated': 'Cropland_Intense',
    'Transformation, to annual crop, irrigated, extensive': 'Cropland_Light',
    'Transformation, to annual crop, irrigated, intensive': 'Cropland_Intense',
    'Transformation, to annual crop, non-irrigated': 'Cropland_Intense',
    'Transformation, to annual crop, non-irrigated, extensive': 'Cropland_Minimal',
    'Transformation, to annual crop, non-irrigated, intensive': 'Cropland_Intense',
    'Transformation, to arable land, unspecified use': 'Cropland_Intense',
    # 'Transformation, to bare area (non-use)': '',
    'Transformation, to cropland fallow (non-use)': 'Cropland_Minimal',
    'Transformation, to dump site': 'Urban_Intense',
    'Transformation, to dump site, inert material landfill': 'Urban_Intense',
    'Transformation, to dump site, residual material landfill': 'Urban_Intense',
    'Transformation, to dump site, sanitary landfill': 'Urban_Intense',
    'Transformation, to dump site, slag compartment': 'Urban_Intense',
    'Transformation, to field margin/hedgerow': 'Cropland_Minimal',
    'Transformation, to forest, extensive': 'Managed_forest_Light',
    'Transformation, to forest, intensive': 'Managed_forest_Intense',
    'Transformation, to forest, primary (non-use)': 'Pasture_Minimal',
    'Transformation, to forest, secondary (non-use)': 'Managed_forest_Light',
    'Transformation, to forest, unspecified': 'Managed_forest_Intense',
    'Transformation, to grassland, natural (non-use)': 'Pasture_Minimal',
    'Transformation, to grassland, natural, for livestock grazing': 'Pasture_Minimal',
    'Transformation, to heterogeneous, agricultural': 'Cropland_Minimal',
    'Transformation, to industrial area': 'Urban_Intense',
    # 'Transformation, to inland waterbody, unspecified': '',
    # 'Transformation, to lake, artificial': '',
    # 'Transformation, to lake, natural (non-use)': '',
    'Transformation, to mineral extraction site': 'Urban_Intense',
    'Transformation, to pasture, man made': 'Pasture_Intense',
    'Transformation, to pasture, man made, extensive': 'Pasture_Light',
    'Transformation, to pasture, man made, intensive': 'Pasture_Intense',
    'Transformation, to permanent crop': 'Cropland_Intense',
    'Transformation, to permanent crop, irrigated': 'Cropland_Intense',
    'Transformation, to permanent crop, irrigated, extensive': 'Cropland_Light',
    'Transformation, to permanent crop, irrigated, intensive': 'Cropland_Intense',
    'Transformation, to permanent crop, non-irrigated': 'Cropland_Intense',
    'Transformation, to permanent crop, non-irrigated, extensive': 'Cropland_Minimal',
    'Transformation, to permanent crop, non-irrigated, intensive': 'Cropland_Intense',
    # 'Transformation, to river, artificial': '',
    # 'Transformation, to river, natural (non-use)': '',
    # 'Transformation, to seabed, drilling and mining': '',
    # 'Transformation, to seabed, infrastructure': '',
    # 'Transformation, to seabed, natural (non-use)': '',
    # 'Transformation, to seabed, unspecified': '',
    'Transformation, to shrub land, sclerophyllous': 'Pasture_Minimal',
    # 'Transformation, to snow and ice (non-use)': '',
    'Transformation, to traffic area, rail network': 'Urban_Intense',
    'Transformation, to traffic area, rail/road embankment': 'Urban_Light',
    'Transformation, to traffic area, road network': 'Urban_Intense',
    'Transformation, to unknown': 'Urban_Intense', # Added - Matched to unspecified
    'Transformation, to unspecified': 'Urban_Intense',
    'Transformation, to unspecified, natural (non-use)': 'Pasture_Minimal',
    'Transformation, to urban, continuously built': 'Urban_Intense',
    'Transformation, to urban, discontinuously built': 'Urban_Light',
    'Transformation, to urban, green area': 'Urban_Minimal',
    'Transformation, to urban/industrial fallow (non-use)': 'Urban_Light',
    # 'Transformation, to wetland, coastal (non-use)': '',
    # 'Transformation, to wetland, inland (non-use)': '',   
}

In [25]:
transformation_from_flows = {
    'Transformation, from annual crop': 'Cropland_Intense',
    'Transformation, from annual crop, flooded crop': 'Cropland_Intense',
    'Transformation, from annual crop, greenhouse': 'Urban_Light',
    'Transformation, from annual crop, irrigated': 'Cropland_Intense',
    'Transformation, from annual crop, irrigated, extensive': 'Cropland_Light',
    'Transformation, from annual crop, irrigated, intensive': 'Cropland_Intense',
    'Transformation, from annual crop, non-irrigated': 'Cropland_Intense',
    'Transformation, from annual crop, non-irrigated, extensive': 'Cropland_Minimal',
    'Transformation, from annual crop, non-irrigated, intensive': 'Cropland_Intense',
    'Transformation, from arable land, unspecified use': 'Cropland_Intense',
    # 'Transformation, from bare area (non-use)': '',
    'Transformation, from cropland fallow (non-use)': 'Cropland_Minimal',
    'Transformation, from dump site': 'Urban_Intense',
    'Transformation, from dump site, inert material landfill': 'Urban_Intense',
    'Transformation, from dump site, residual material landfill': 'Urban_Intense',
    'Transformation, from dump site, sanitary landfill': 'Urban_Intense',
    'Transformation, from dump site, slag compartment': 'Urban_Intense',
    'Transformation, from field margin/hedgerow': 'Cropland_Minimal',
    'Transformation, from forest, extensive': 'Managed_forest_Light',
    'Transformation, from forest, intensive': 'Managed_forest_Intense',
    'Transformation, from forest, primary (non-use)': 'Pasture_Minimal',
    'Transformation, from forest, secondary (non-use)': 'Managed_forest_Light',
    'Transformation, from forest, unspecified': 'Managed_forest_Intense',
    'Transformation, from grassland, natural (non-use)': 'Pasture_Minimal',
    'Transformation, from grassland, natural, for livestock grazing': 'Pasture_Minimal',
    'Transformation, from heterogeneous, agricultural': 'Cropland_Minimal',
    'Transformation, from industrial area': 'Urban_Intense',
    # 'Transformation, from inland waterbody, unspecified': '',
    # 'Transformation, from lake, artificial': '',
    # 'Transformation, from lake, natural (non-use)': '',
    'Transformation, from mineral extraction site': 'Urban_Intense',
    'Transformation, from pasture, man made': 'Pasture_Intense',
    'Transformation, from pasture, man made, extensive': 'Pasture_Light',
    'Transformation, from pasture, man made, intensive': 'Pasture_Intense',
    'Transformation, from permanent crop': 'Cropland_Intense',
    'Transformation, from permanent crop, irrigated': 'Cropland_Intense',
    'Transformation, from permanent crop, irrigated, extensive': 'Cropland_Light',
    'Transformation, from permanent crop, irrigated, intensive': 'Cropland_Intense',
    'Transformation, from permanent crop, non-irrigated': 'Cropland_Intense',
    'Transformation, from permanent crop, non-irrigated, extensive': 'Cropland_Minimal',
    'Transformation, from permanent crop, non-irrigated, intensive': 'Cropland_Intense',
    # 'Transformation, from river, artificial': '',
    # 'Transformation, from river, natural (non-use)': '',
    # 'Transformation, from seabed, drilling and mining': '',
    # 'Transformation, from seabed, infrastructure': '',
    # 'Transformation, from seabed, natural (non-use)': '',
    # 'Transformation, from seabed, unspecified': '',
    'Transformation, from shrub land, sclerophyllous': 'Pasture_Minimal',
    # 'Transformation, from snow and ice (non-use)': '',
    'Transformation, from traffic area, rail network': 'Urban_Intense',
    'Transformation, from traffic area, rail/road embankment': 'Urban_Light',
    'Transformation, from traffic area, road network': 'Urban_Intense',
    'Transformation, from unknown': 'Urban_Intense', # Added - Matched to unspecified
    'Transformation, from unspecified': 'Urban_Intense',
    'Transformation, from unspecified, natural (non-use)': 'Pasture_Minimal',
    'Transformation, from urban, continuously built': 'Urban_Intense',
    'Transformation, from urban, discontinuously built': 'Urban_Light',
    'Transformation, from urban, green area': 'Urban_Minimal',
    'Transformation, from urban/industrial fallow (non-use)': 'Urban_Light',
    # 'Transformation, from wetland, coastal (non-use)': '',
    # 'Transformation, from wetland, inland (non-use)': '',   
}

# 6. JSON generation

In [26]:
cc = coco.CountryConverter()
species_groups = final_output['species_group'].unique()
print(f"Found {len(species_groups)} species groups: {species_groups}")

Found 1 species groups: ['Eukaryota']


In [ ]:
def generate_occupation_json(species_group):
    """Generate JSON for occupation flows for a specific species group"""
    
    exchanges = []
    
    # Group by country and biosphere flow
    for flow_name, habitat_type in occupation_flows.items():
        
        # Get all countries
        countries_iso_list = final_output['country_iso'].unique()
        
        for country in countries_iso_list:
            # Get country-level CF from cfs_country for this species group
            country_cf_row = cfs_country[
                (cfs_country['iso3cd'] == country) &
                (cfs_country['habitat'] == habitat_type) &
                (cfs_country['species_group'] == species_group)
            ]
            
            # If no country-level CF available, skip this country-habitat combination
            if len(country_cf_row) == 0:
                continue
            
            # Get the country-level CF value
            country_cf_value = country_cf_row['CF_occ_avg_glo'].iloc[0]
            
            # Filter ecoregion data for uncertainty parameters
            country_data = final_output[
                (final_output['country_iso'] == country) &
                (final_output['habitat_type'] == habitat_type) &
                (final_output['species_group'] == species_group)
            ]
            
            if len(country_data) == 0:
                continue
            
            # Get unique ecoregions for uncertainty
            unique_ecoregions = country_data.drop_duplicates(subset=['ecoregion_id'])
            
            # Prepare uncertainty parameters from ecoregion data
            values = unique_ecoregions['cf_occupation_avg_global'].tolist()
            weights = unique_ecoregions['weight_factor'].tolist()
            
            # Get country revenue
            country_revenue_lookup = country_totals_list.set_index('ISO_A3')['country_total'].to_dict()
            country_revenue = country_revenue_lookup.get(country, 0.0)
            
            exchange = {
                "supplier": {
                    "name": flow_name,
                    "categories": ["natural resource", "land"],
                    "matrix": "biosphere"
                },
                "consumer": {
                    "location": cc.convert(names=country, to='ISO2'),
                    "matrix": "technosphere"
                },
                "value": float(country_cf_value),
                "weight": float(country_revenue),
                "uncertainty": {
                    "distribution": "discrete_empirical",
                    "parameters": {
                        "values": [float(v) for v in values],
                        "weights": [float(w) for w in weights]
                    }
                }
            }
            
            exchanges.append(exchange)
    
    output = {
        "name": f"ecoinvent 3.10/3.11 - GLAM3 - Land occupation, biodiversity damage - {species_group}",
        "unit": "PDF.yr",
        "version": "0.1",
        "description": f"Characterization factors for land occupation impacts on biodiversity ({species_group}), considering global species loss. Based on Scherer et al. 2023: https://doi.org/10.5281/zenodo.10114492",
        "species_group": species_group,
        "strategies": [
            "map_exchanges",
            "map_aggregate_locations",
            "map_dynamic_locations",
            "map_contained_locations",
            "map_remaining_locations_to_global"
        ],
        "exchanges": exchanges
    }
    
    return output

In [28]:
def generate_transformation_json(species_group):
    """Generate JSON for transformation flows (from and to combined) for a specific species group"""
    
    exchanges = []
    
    # Process transformation FROM flows
    for flow_name, habitat_type in transformation_from_flows.items():
        
        countries_iso_list = final_output['country_iso'].unique()
        
        for country in countries_iso_list:
            # Get country-level CF from cfs_country for this species group
            country_cf_row = cfs_country[
                (cfs_country['iso3cd'] == country) &
                (cfs_country['habitat'] == habitat_type) &
                (cfs_country['species_group'] == species_group)
            ]
            
            if len(country_cf_row) == 0:
                continue
            
            # Get the country-level CF value for transformation
            country_cf_value = country_cf_row['CF_tra_avg_glo'].iloc[0]
            
            # Filter ecoregion data for uncertainty parameters
            country_data = final_output[
                (final_output['country_iso'] == country) &
                (final_output['habitat_type'] == habitat_type) &
                (final_output['species_group'] == species_group)
            ]
            
            if len(country_data) == 0:
                continue
            
            unique_ecoregions = country_data.drop_duplicates(subset=['ecoregion_id'])
            
            values = unique_ecoregions['cf_transformation_avg_global'].tolist()
            weights = unique_ecoregions['weight_factor'].tolist()
            
            country_revenue_lookup = country_totals_list.set_index('ISO_A3')['country_total'].to_dict()
            country_revenue = country_revenue_lookup.get(country, 0.0)            
            
            exchange = {
                "supplier": {
                    "name": flow_name,
                    "categories": ["natural resource", "land"],
                    "matrix": "biosphere"
                },
                "consumer": {
                    "location": cc.convert(names=country, to='ISO2'),
                    "matrix": "technosphere"
                },
                "value": float(country_cf_value),
                "weight": float(country_revenue),
                "uncertainty": {
                    "distribution": "discrete_empirical",
                    "parameters": {
                        "values": [float(v) for v in values],
                        "weights": [float(w) for w in weights]
                    }
                }
            }
            
            exchanges.append(exchange)
    
    # Process transformation TO flows
    for flow_name, habitat_type in transformation_to_flows.items():
        
        countries_iso_list = final_output['country_iso'].unique()
        
        for country in countries_iso_list:
            # Get country-level CF from cfs_country for this species group
            country_cf_row = cfs_country[
                (cfs_country['iso3cd'] == country) &
                (cfs_country['habitat'] == habitat_type) &
                (cfs_country['species_group'] == species_group)
            ]
            
            if len(country_cf_row) == 0:
                continue
            
            # Get the country-level CF value for transformation
            country_cf_value = country_cf_row['CF_tra_avg_glo'].iloc[0]
            
            # Filter ecoregion data for uncertainty parameters
            country_data = final_output[
                (final_output['country_iso'] == country) &
                (final_output['habitat_type'] == habitat_type) &
                (final_output['species_group'] == species_group)
            ]
            
            if len(country_data) == 0:
                continue
            
            unique_ecoregions = country_data.drop_duplicates(subset=['ecoregion_id'])
            
            values = unique_ecoregions['cf_transformation_avg_global'].tolist()
            weights = unique_ecoregions['weight_factor'].tolist()
            
            country_revenue_lookup = country_totals_list.set_index('ISO_A3')['country_total'].to_dict()
            country_revenue = country_revenue_lookup.get(country, 0.0)
            
            exchange = {
                "supplier": {
                    "name": flow_name,
                    "categories": ["natural resource", "land"],
                    "matrix": "biosphere"
                },
                "consumer": {
                    "location": cc.convert(names=country, to='ISO2'),
                    "matrix": "technosphere"
                },
                "value": float(country_cf_value),
                "weight": float(country_revenue),
                "uncertainty": {
                    "distribution": "discrete_empirical",
                    "parameters": {
                        "values": [float(v) for v in values],
                        "weights": [float(w) for w in weights]
                    }
                }
            }
            
            exchanges.append(exchange)
    
    output = {
        "name": f"ecoinvent 3.10/3.11 - GLAM3 - Land transformation, biodiversity damage - {species_group}",
        "unit": "PDF.yr",
        "version": "0.1",
        "description": f"Characterization factors for land transformation impacts on biodiversity ({species_group}), considering global species loss. Based on Scherer et al. 2023: https://doi.org/10.5281/zenodo.10114492",
        "species_group": species_group,
        "strategies": [
            "map_exchanges",
            "map_aggregate_locations",
            "map_dynamic_locations",
            "map_contained_locations",
            "map_remaining_locations_to_global"
        ],
        "exchanges": exchanges
    }
    
    return output

In [30]:
for species_group in species_groups:
    print(f"\n--- Processing {species_group} ---")
    
    # Generate occupation JSON for this species group
    print(f"\n=== Generating Occupation JSON for {species_group} ===")
    occupation_json = generate_occupation_json(species_group)
    
    # Generate transformation JSON for this species group
    print(f"\n=== Generating Transformation JSON for {species_group} ===")
    transformation_json = generate_transformation_json(species_group)
    
    # Save to files with species group in filename
    species_group_clean = species_group.replace(' ', '_').lower()
    
    occupation_filename = f"output/test_severin/test_severin_GLAM3_biodiversity_occupation_average_{species_group_clean}.json"
    transformation_filename = f"output/test_severin/test_severin_GLAM3_biodiversity_transformation_average_{species_group_clean}.json"
    
    with open(occupation_filename, "w") as f:
        json.dump(occupation_json, f, indent=2)
    
    with open(transformation_filename, "w") as f:
        json.dump(transformation_json, f, indent=2)


--- Processing Eukaryota ---

=== Generating Occupation JSON for Eukaryota ===

=== Generating Transformation JSON for Eukaryota ===
